In [ ]:
# Install required libraries
#!pip install tensorflow transformers scikit-learn spacy

import json
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import spacy
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random
from tensorflow.keras.models import load_model
import pickle

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Function to load JSON data
def load_json(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    return data

# Function to train the neural network model
def train_model(train_x, train_y):
    # Create NN model to predict the responses
    model_nn = Sequential()
    model_nn.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(64, activation='relu'))
    model_nn.add(Dropout(0.5))
    model_nn.add(Dense(len(train_y[0]), activation='softmax'))

    # Compile model using the newer version of SGD optimizer
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model_nn.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # Fitting and saving the model
    hist = model_nn.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
    model_nn.save('chatbot.h5')  # Save the model for future use
    print("\n")
    print("*" * 50)
    print("\nModel Created Successfully!")

# Function to score answer relevancy using TF-IDF
def score_relevancy(user_input, candidate_answers):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([user_input] + candidate_answers)
    similarity_matrix = cosine_similarity(vectors)
    relevancy_scores = similarity_matrix[0][1:]
    return relevancy_scores

# Function to extract keywords using spaCy
def extract_keywords(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc if token.is_alpha]

# Function to handle user input
def handle_user_input(user_input, technical_ability, test_data):
    user_keywords = extract_keywords(user_input)
    print(f"User Keywords: {user_keywords}")

    for example in test_data["intents"]:
        if "question" in example:
            question_keywords = [keyword for q in example["question"] for keyword in extract_keywords(q)]

            if all(keyword in question_keywords for keyword in user_keywords):
                answer = example.get("answer", [])
                return f"Answer from JSON file: {answer}"

    input_ids = tokenizer.encode(user_input, return_tensors="tf")
    output = model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Adjust the response based on the user's technical ability
    if technical_ability == "tech":
        # Customize response for technical users
        pass
    else:
        # Customize response for non-technical users
        pass

    candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
    relevancy_scores = score_relevancy(user_input, candidate_answers)

    # Store or update relevancy scores for future learning
    # Update your model based on user feedback and learning algorithm

    return bot_response

# Load or collect data for training/fine-tuning
data = load_json("/content/tech (1).json")

# Example data preparation (replace this with your actual training data)
# For now, using placeholder values
train_x = np.random.rand(10, 5)  # Placeholder values for training features
train_y = np.random.randint(2, size=(10, 3))  # Placeholder values for training labels

# Train or fine-tune the model
train_model(train_x, train_y)

# Example usage
while True:
    user_input = input("Enter the Question you want (type 'break' or 'quit' to exit): ")
    if user_input.lower() in ["break", "quit"]:
        break
    technical_ability = input("Enter the user's technical ability: ")
    response = handle_user_input(user_input, technical_ability, data)
    print(response)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Epoch 1/200
2/2 [==============================] - 2s 18ms/step - loss: 1.8879 - accuracy: 0.5000
Epoch 2/200
2/2 [==============================] - 0s 11ms/step - loss: 1.8315 - accuracy: 0.6000
Epoch 3/200
2/2 [==============================] - 0s 20ms/step - loss: 1.8459 - accuracy: 0.8000
Epoch 4/200
2/2 [==============================] - 0s 20ms/step - loss: 1.7456 - accuracy: 0.9000
Epoch 5/200
2/2 [==============================] - 0s 15ms/step - loss: 1.8018 - accuracy: 0.9000
Epoch 6/200
2/2 [==============================] - 0s 13ms/step - loss: 1.8461 - accuracy: 0.8000
Epoch 7/200
2/2 [==============================] - 0s 14ms/step - loss: 1.7967 - accuracy: 0.9000
Epoch 8/200
2/2 [==============================] - 0s 11ms/step - loss: 1.8160 - accuracy: 0.9000
Epoch 9/200
2/2 [==============================] - 0s 10ms/step - loss: 1.7239 - accuracy: 0.9000
Epoch 10/200
2/2 [==============================] - 0s 15ms/step - loss: 1.7577 - accuracy: 0.9000
Epoch 11/200
2/2 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(




**************************************************

Model Created Successfully!
Enter the Question you want (type 'break' or 'quit' to exit): hi
Enter the user's technical ability: non
User Keywords: ['hi']
Answer from JSON file: ['Hello! how can i help you ?']
Enter the Question you want (type 'break' or 'quit' to exit): what is ai
Enter the user's technical ability: tech
User Keywords: ['what', 'is', 'ai']
Answer from JSON file: ['AI is the development of computer systems that can perform tasks requiring human intelligence.']
Enter the Question you want (type 'break' or 'quit' to exit): break


In [ ]:
from graphviz import Digraph

code = """# full translation not working
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right',font=('Times New Roman', 14, 'bold'), background='lightgreen')
        self.textcon.tag_config('bot', foreground='black', justify='left',font=('Times New Roman', 14, 'bold'), background='lightblue')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 14))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 14, "bold"))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")
        self.textcon.see(tk.END)

    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'Friday: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'Friday', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'Friday', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("Friday: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"Friday: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'Friday', self.bot_icon)

        # Check if the translation process is already ongoing
        if not hasattr(self, 'translation_in_progress'):
            # Insert bot response
            self.insert_message('bot', 'Available Languages:')
            # Insert available languages in the GUI
            for code, lang in languages.items():
                self.insert_message('bot', f'{code}: {lang}')

            mic = sr.Microphone()
            r = sr.Recognizer()

            with mic as source:
                translator = Translator()

                # Get input language from user using simpledialog
                input_language = simpledialog.askstring("Input Language", "Select Language from:")
                # Get output language from user using simpledialog
                output_language = simpledialog.askstring("Output Language", "Select your Language To:")

                print("Speak the text that needs to be translated ..")
                r.adjust_for_ambient_noise(source, duration=0.2)
                audio3 = r.listen(source)
                text3 = r.recognize_google(audio3)

                print("You said: " + text3.lower())

                # Translate the user's question
                translated_question = translator.translate(text3, src=input_language, dest=output_language).text.lower()

                # Update GUI with the translation
                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', f'Translation: {translated_question}')
                if hasattr(self, 'input_source') and self.input_source == 'mic':
                    self.speak(translated_question)
                self.textcon.insert(tk.END, "\n")

                # Set the variable to indicate that translation is in progress
                self.translation_in_progress = True
                return  # Exit the function to avoid rerunning the translation process

        # If translation process is already in progress, clear the variable
        self.translation_in_progress = False


    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()
 """


from graphviz import Digraph

def create_flowchart():
    dot = Digraph(comment='The Flowchart')

    dot.node('A', 'Start')
    dot.node('B', 'Greeting User')
    dot.node('C', 'User input')
    dot.node('D', 'Answer from json')
    dot.node('E', 'Answer from GPT2 Model')
    dot.node('F', 'Play video')
    dot.node('G', 'Search Google')
    dot.node('H', 'Translate')
    dot.node('I', 'End')
    dot.node('J', 'Loop Start', shape='diamond', color='orange')
    dot.node('K', 'Loop End', shape='diamond', color='orange')

    dot.edges(['AB', 'BC'])
    dot.edge('C', 'D', label='input found in json')
    dot.edge('C', 'E', label='Not found in json')
    dot.edge('C', 'F', label='input is play')
    dot.edge('C', 'G', label='input is search')
    dot.edge('C', 'H', label='input is translate')
    dot.edge('D', 'J', label='Answer found in json')
    dot.edge('E', 'J', label='Answer from GPT2 Model')
    dot.edge('F', 'J', label='Play video')
    dot.edge('G', 'J', label='Search Google')
    dot.edge('H', 'J', label='Translate')

    dot.edge('J', 'K', label='User input is not bye, goodbye, or off', constraint='false')
    dot.edge('K', 'C')

    dot.edge('J', 'I', label='User input is bye, goodbye, or off', constraint='true')

    return dot

if __name__ == '__main__':
    flowchart = create_flowchart()
    flowchart.render('flowchart_with_loop', format='png', cleanup=True)
    flowchart.view()
